# Acquired equivalence

I think the AE task usually involves associating stimuli that lead to the same outcome, which is usually a reward or something like that. So this is not really that. This is like a paired associate inference task, but with a slightly different structure:

Learn a-b, c-b (PAI is learn a-b, b-c)

Test: a vs. c/d?

In [1]:
list_length = 50
interjection_length = 200

import random
import numpy as np
import matplotlib.pyplot as plt
from helpers import * # general helper functions
%reload_ext autoreload
%autoreload 2

## task-specific helper functions

In [2]:
def plot_results(target_words, lure_words, output_words, interjected_list=[]):
    recall = [w in output_words for w in target_words]
    precision = [w in set(target_words) for w in np.unique(output_words)]
    precision_lure = [w in set(lure_words) for w in np.unique(output_words)]
    precision_int = [w in set(interjected_list) for w in np.unique(output_words)]
    
    ncols = 3 if len(interjected_list) == 0 else 4
    fig,ax = plt.subplots(1,ncols)
    ax[0].plot(recall)
    ax[0].set_title('Recall')
    ax[0].set_xlabel('Word in true list')
    ax[0].set_ylabel('Present in recalled list')

    ax[1].plot(precision)
    ax[1].set_title('Precision')
    ax[1].set_xlabel('Word in output (unique)')
    ax[1].set_ylabel('Present in TRUE list')

    ax[2].plot(precision_lure)
    ax[2].set_title('Lure precision')
    ax[2].set_xlabel('Word in output (unique)')
    ax[2].set_ylabel('Present in LURE list')

    if len(interjected_list) > 0:
        ax[3].plot(precision_int)
        ax[3].set_title('Interjection precision')
        ax[3].set_xlabel('Word in output (unique)')
        ax[3].set_ylabel('Present in INTERJECTION list')

## define task

In [3]:
list_length = round(list_length/3)*3 # make into closest multiple of 3

all_words = txt_to_list('_data/wasnorm_wordpool.txt')

# trials are either a->b, c->b, or d->e
# 1/3 of trials have d, 1/3 have e, 1/3 a, 1/3 c, 2/3 b
words = subsample_words(all_words, list_length*2, seed=0)
a_words = words[:int(list_length/3)]
b_words = words[int(list_length/3):list_length]
c_words = words[list_length:int(4/3*list_length)]
d_words = words[int(4/3*list_length):int(5/3*list_length)]
e_words = words[int(5/3*list_length):]

ab_pairs = make_pairs(a_words, b_words) # 1/3 of trials
cb_pairs = make_pairs(c_words,b_words) # 1/3 of trials
de_pairs = make_pairs(d_words, e_words) # 1/3 of trials

# AB and CB pairs can come in any order; just pure randomize
training_pairs = make_training_order(ab_pairs, cb_pairs, de_pairs)

test_probes, targets, lures = make_test_probes(cue_words = a_words, target_words = c_words, lure_words = e_words)

interjection_words = subsample_words(all_words, interjection_length*2, seed=0, avoid=words)
interjection_pairs = make_pairs(interjection_words[:interjection_length], interjection_words[interjection_length:])

In [5]:
ae_inp = make_inp(
    training_pairs, test_probes,
    preamble = "Here is a list of word pairs:",
    cue = "Now we'll begin the test. " \
        "Here is a list of test probes; each one contains a CUE word, from the original list, and two possible TARGETS. " \
        "Select a target based on the associations learned in the original list. " \
        "Please output a list of your chosen targets (one of the two target words, for each cue in the list); do not respond with any word that is not one of the possible targets. " \
        "Do not think step by step, only output the list of chosen targets. Here are the test words:",
)
print(ae_inp)

ae_inp_informed = make_inp(
    training_pairs, test_probes,
    preamble = "Here is a list of word pairs. " \
        "After seeing this list, you will be tested on inferential associations between words in this list. " \
        "Here are the word pairs:",
    cue = "Now we'll begin the test. " \
        "Here is a list of test probes; each one contains a CUE word, from the original list, and two possible TARGETS. " \
        "Select a target based on the associations learned in the original list. " \
        "Please output a list of your chosen targets (one of the two target words, for each cue in the list); do not respond with any word that is not one of the possible targets. " \
        "Do not think step by step, only output the list of chosen targets. Here are the test words:",
)
print(ae_inp_informed)

ae_inp_informed_interjected = make_inp(
    training_pairs, test_probes,
    preamble = "Here is a list of word pairs. " \
        "After seeing this list, you will see another list of pairs. " \
        "Then, you will be tested on inferential associations between words in this original list of pairs. " \
        "Here are the word pairs:",
    cue = "Now we'll begin the test. " \
        "Here is a list of test probes; each one contains a CUE word, from the original list, and two possible TARGETS. " \
        "Select a target based on the associations learned in the original list. " \
        "Please output a list of your chosen targets (one of the two target words, for each cue in the list); do not respond with any word that is not one of the possible targets. " \
        "Do not think step by step, only output the list of chosen targets. Here are the test words:",
    interjection = "You will now see another list of word pairs. Later, during the test, you should not base your responses on the pairs in this list:",
    interjection_words = interjection_pairs,
)
print(ae_inp_informed_interjected)

Here is a list of word pairs: roach-traitor, paint-sage, grasshopper-monster, pen-buggy, list-telephone, perch-earring, chemist-crutch, vehicle-van, priest-country, brain-foot, knuckle-cheek, employee-zucchini, dough-porcupine, pearl-sugar, hamper-costume, camera-zebra, bug-rodent, igloo-butcher, teapot-traitor, lid-cheek, cheddar-hall, onion-giraffe, shell-zucchini, temple-fort, bank-senate, gallon-gorilla, garbage-chimney, stallion-senate, fountain-dresser, ozone-plumber, postage-saturn, thicket-van, guard-butcher, whistle-sap, producer-refrigerator, deodorant-boss, moss-president, screw-plumber, acorn-earring, bird-boss, hand-sap, transplant-giraffe, cousin-sugar, raisin-hall, fleet-oatmeal, man-bulletin, detergent-oatmeal, driveway-chimney, castle-copier, widow-bulletin, paper-telephone. Now we'll begin the test. Here is a list of test probes; each one contains a CUE word, from the original list, and two possible TARGETS. Select a target based on the associations learned in the ori

## run task

In [ ]:
pipe = make_pipe(model_id = "Qwen/Qwen3-4B-Instruct-2507")

In [ ]:
for inp in [ae_inp, ae_inp_informed, ae_inp_informed_interjected]:
    print(inp)
    output = query_model(pipe, inp)
    plot_results(targets, lures, output, interjection_words) # C are targets, E are lures